Network Science
===========================



This tutorial gives an introduction to Network Science.


Please use **Python3** for your default kernel this will not work otherwise!

If you are using google colab this should just work :-)

For those who are new to colabs/Jupyter:


shift-enter - Executes a cell

alt-enter   -  inserts a cell below


## Preamble

This is needed for Colab, but if you are running this in Jupyter, please install your packages separately

In [0]:
%load_ext autoreload
%autoreload 2
# install im_tutorial package
!pip install git+https://github.com/nestauk/im_tutorials.git
!pip install pybind11
!pip install cpalgorithm
!pip install networkx
!pip install seaborn
# Needed to make some of the drag and dropping work
!pip install -U bokeh

In [0]:
# importing useful Python utility libraries we'll need
from collections import Counter, defaultdict
import itertools

# matplotlib for static plots
import matplotlib.pyplot as plt
from matplotlib import pylab

# makes the plots prettier
import seaborn

# numpy for mathematical functions
import numpy as np

# pandas for handling tabular data
import pandas as pd

# networkx for the analysis of graphs
import networkx as nx

#from im_tutorials.utilities import chunks
from im_tutorials.data import *

# Useful data structure to count occurances
from collections import Counter

#Iteration lib
import itertools as it

### Interactive Network Visualisation
To visualise our networks we will use a nice library called bokeh. This will be covered in depth in a later session, so for now let us just assume that it is a magic, and all will be revealed later.

In [0]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Category20, Spectral4
from bokeh.palettes import Spectral6
from bokeh.models import Circle, MultiLine, HoverTool, TapTool, BoxSelectTool
from bokeh.models import ColumnDataSource, LinearColorMapper, PointDrawTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges
output_notebook()

Now we have everything we need to make a nice plot. Luckily, `bokeh` has built-in support for `networkx` graphs, which makes plotting and interacting with them easy.

In [0]:
from bokeh.palettes import Spectral4
from bokeh.transform import factor_cmap
palette = seaborn.cubehelix_palette(21)
pal_hex_lst = palette.as_hex()
def bokeh_plot(G,attribute=None,title='',factor=False):

  # Create a plot and give it some basic features.
  plot = figure(title=title,
              x_range=(-2.1,2.1), y_range=(-2.1,2.1),
             )
  pos = nx.spring_layout(G, weight='association_strength', scale=2)
  # Use the renderer built in to `bokeh` to transform our Graph
  # object into something that `bokeh` can plot.
  graph_renderer = from_networkx(G, pos, center=(0,0))
  
  # Draw glyphs for our nodes and assign properties for interactions.
  if attribute!=None:
    idx = graph_renderer.node_renderer.data_source.data['index']
    data = [attribute[x] for x in idx]
    graph_renderer.node_renderer.data_source.data['node_color'] = data

    if factor:
      data = [str(attribute[x]) for x in idx]
      graph_renderer.node_renderer.data_source.data['node_color'] = data
      uniItems = list(set(data))
      uniItemsNum = min(20,len(uniItems))
      mapper = factor_cmap('node_color', palette=Category20[uniItemsNum], factors=uniItems)
      graph_renderer.node_renderer.glyph = Circle(size=10,fill_color= mapper) #, line_color=None)

    else:
      mapper = LinearColorMapper(palette=pal_hex_lst, low=min(attribute.values()), high=max(attribute.values()))
      graph_renderer.node_renderer.glyph = Circle(size=10,fill_color={'field': 'node_color', 'transform': mapper}) #, line_color=None)
  else:
    graph_renderer.node_renderer.glyph = Circle(size=10,fill_color='blue') #, line_color=None)

    
  graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
  graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])                               
  graph_renderer.node_renderer.muted_glyph = Circle(size=15, fill_alpha=0.9)
  
  # Draw glyphs for edges and assign properties for interactions.
  graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=1, line_width=1)
  graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=1.5)
  graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=1.5)
  
  # Add the ability to select nodes.
  graph_renderer.selection_policy = NodesAndLinkedEdges()
  # Add a hover tool, that allows us to investigate nodes with a tooltip. 
  hover = HoverTool(tooltips=[("Name:", "@name")])
  
  
  # Put everything on the plot.
  graph_renderer.node_renderer.data_source.data['name'] =graph_renderer.node_renderer.data_source.data['index']

  fill_color='color'
  plot.add_tools(hover, TapTool(), BoxSelectTool())
  plot.renderers.append(graph_renderer)

  show(plot)

  # Uncomment this line if using google colab
  output_notebook()

## Lets Load Some Data


We will start with some simple data from Les Miserable. It is a weighted undirected graph, where the weights are the number of coappearances (and the nodes are characters in the novel).

If you wanted to run the same code with a different graph you would just need to place your graph here :-), although if the graph more than a few hundred nodes you might wait a while for the plotting commands so it might be worth skipping them :-).

In [0]:
G = nx.les_miserables_graph()

To see the nodes and edges we use the following commands:

In [0]:
print(G.nodes())

In [0]:
print(G.edges())

To look at the edge weights we use the following

In [0]:
print([(x,y,G[x][y]['weight']) for x in G for y in G[x] if x<y])

We see that the edge weights vary quite a lot. For this simple example we will remove the weights to give us a undirected and unweighted graph.

In [0]:
G = nx.les_miserables_graph()
for x in G:
    for y in G[x]:
      if x<y:
        del G[x][y]['weight']

Lets imagine we have never seen this network before.

Let us follow the lectures notes:

1. Look at some basic measurements
2. Look at if the network is connected.
3. Look at the degree distribution
4. Visualise it :-)





Some basic summary statistics:

In [0]:
print(['Num_Nodes', G.number_of_nodes()])
print(['Num_Edges', G.number_of_edges()])
print(['Num Trianges', sum(nx.triangles(G).values())/3])
print(['Clustering Coef.', nx.transitivity(G)])

Lets check if the network is connected:

In [0]:
print(nx.number_connected_components(G))

Great only one component!

Lets check the degree distribution:

In [0]:
degrees = dict(nx.degree(G))
pylab.hist(list(dict(degrees).values()),10)

Finally draw the graph:

In [0]:
## Really simple network diagram so we can see what we are dealing with
nx.draw(G)

But we can do a little better than that, using the magic bokeh package:

(try hovering over and selecting the nodes)



In [0]:
bokeh_plot(G)

Network Centrality Measures
===========================


As we discussed in the lecture, centrality measures are a set of statistics that attempt to capture who/what are the most important nodes in a network.

However there are many different ways to do it. Here we are going to talk through a few very common ones.

-   Probably hundreds of different types of centralities, though many of
    them are very similar to each other


We will only discuss a few well-known examples

-   The simplest type of centrality is Degree Centrality

-   Closeness Centrality

-   Betweenness Centrality

-   Eigenvector Centrality

-   Katz Centrality

-   Page Rank algorithm


### Background (from slides)

-   Mostly tools from Social Network Analysis

-   Attempt to quantify the importance of nodes, edges, or other network
    structures in various ways

-   The choice depends on the problem and data under study, as what it
    means to be "most central\" (and most important) is obviously
    context-dependent

-   Various notions/concepts of centrality (vary by context and purpose)

-   Centralities are often not robust to small perturbations either of
    their definition or of network structure

-   Be cautious about interpreting the results of centrality
    calculations




### Simplest centrality: Degree Centrality




Lets first look at degree centrality.

First question what is degree centrality? Do you remember?

#### Answer




Just the number of connections that a node has :-)

#### Degree Centrality Continued


In [0]:
degrees = dict(nx.degree(G))
sorted(degrees.items(),key=lambda x:-x[1])[:10]


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Gavroche_%28Les_Misérables%29.jpg/162px-Gavroche_%28Les_Misérables%29.jpg" style="float: right; height: 300px;">



The most important character is *Valjean*, the main character, not that surprising. 

But the second most important character is *Gavroche*


Is that right? Lets have a look at the graph so we can see what is going on :-)

Note: Colour represents degree centrality - darker colours higher degree centrality.

In [0]:
bokeh_plot(G,degrees,'Les Miserable - Degrees')

### Katz centrality (1953)
Reminder:
-   A variant of eigenvector centrality which allows each vertex a small
    amount of centrality *for free* regardless of its position in the
    network or the centrality of its neighbors
    $$x_i = \alpha \sum_{j=1}^{n} A_{ij} x_j +\beta 
    $$

-   $\alpha, \beta > 0$

-   $x = \alpha A x + \beta \mbox{$1$}$


-   (with $\beta=1$) the Katz centrality $C_k(i)$ of vertex $i$, is
    given by the i-th component of the eigenvector
    $$C_k = (I - \alpha A)^{-1} \mbox{$1$}$$

-   $I$ is the identity matrix, $\mbox{$1$}$ denotes the
    all-ones vector

-   $\alpha \in (0,\lambda)$ is a free parameter governing the balance
    between the eigenvector term and the constant term in
    ([\[katzStart\]](#katzStart){reference-type="ref"
    reference="katzStart"})


In [0]:
katz = nx.centrality.katz_centrality_numpy(G)
top10katz = sorted(katz.items(),key=lambda x:-x[1])[:10]
top10katz


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Gavroche_%28Les_Misérables%29.jpg/162px-Gavroche_%28Les_Misérables%29.jpg" style="float: right; height: 300px;">





In Katz the  most important character is *Gavroche*



Is that right? Lets have a look!




In [0]:
bokeh_plot(G,katz,'Les Miserable - Katz')

Lets try to understand what is going on here :-).

We will first look at the subnetwork that just includes the top 10 nodes. We extract that as follows.

In [0]:
# Get the nodes
top10Nodes = [x[0] for x in top10katz]
# Extract the subgraph
Gsubgraph = G.subgraph(top10Nodes)


Lets now plot the graph, but rather than just giving the line can you write it yourself? 

No problem if not (this is all very new), the answer is hidden below :-)

#### Answer


Well I would do it like this, but there are lots of other ways to do it :-).![alt text](https://)

In [0]:
bokeh_plot(Gsubgraph,katz,'Katz Subgraph')

So we can see the that the top few nodes are almost forming a clique (a fully connected subgraph). We can check this by looking at the density of the subgraph (which is almost 1):




In [0]:
nx.density(Gsubgraph)

Thus, if one of them has a high Katz score, then this is added to the Katz score of each of the other nodes giving them a high Katz score. Continuing this means they all get a high Katz score.

## Page-Rank 

-   Issue with Katz centrality: if a vertex with high Katz centrality
    points to many others then those others also get high centrality

-   A high-centrality vertex (like *Yahoo*) pointing to one million
    others gives *all* one million of them high centrality!

-   Fix: share *Yahoo*'s contribution to the centrality of those one
    million pages to which it is pointing

-   Centrality of vertex $u$ is obtained from that of its neighbors
    proportional to their centrality divided by their out-degree
    $$x_i = \alpha \sum_{j=1}^{n} A_{ij} \frac{x_j}{ {k_j^{out}} } + \beta
    $$

-   Note: vertices with no out-going edges ($k_j^{out}=0$) should
    contribute zero to the centrality of others, so set $k_j^{out}=1$


-   $D$ diagonal matrix with $D_{ii} = \max \{ k_i^{out},1 \}$
    $$x = \beta (I - \alpha A D ^{-1})^{-1} 1 =  \beta D (D - \alpha A)^{-1} 1$$

-   Set $\beta = 1$, just a re-scaling and arrive at the Page-Rank
    centrality $$C_{PR} = D ( D - \alpha A)^{-1} 1$$

-   $\alpha$ governs the balance between the eigenvector term and the
    constant term

-   In practice, Google uses $\alpha = 0.85$, no rigorous theory behind
    this choice


In [0]:
pagerank = nx.pagerank(G)
sorted(pagerank.items(),key=lambda x:-x[1])[:10]



<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Mgr_Bienvenu_par_Gustave_Brion.jpg/144px-Mgr_Bienvenu_par_Gustave_Brion.jpg" style="float: right; height: 300px;">




In pagerank the second most important character is *Marius*



Is that right?



In [0]:
bokeh_plot(G,pagerank,'Les Miserable - PageRank')

So we can see that this has fixed the problem we were seeing in Katz, but again we have to ask ourselves in this the correct choice? 


Do we think that a random walk on this network really makes sense? 


What processes on a social network graph are really like pagerank? 



## Betweenness Centrality

-   The betweenness centrality of an object in a network measures the
    extent to which it lies on short paths

-   A higher betweenness indicates that it lies on more short paths and
    hence should somehow be important for traversing between different
    parts of a network

-   How many pairs of individuals would have to go through you in order
    to reach one another in the minimum number of hops? Who has higher
    betweenness, X or Y?

### Betweenness Centrality of a Vertex

-   The geodesic betweenness $B_{n}(i)$ of a **vertex** in a weighted,
    undirected network is
    $$B_{n}(i) =  \sum_{s,t \in G} \frac{ \Psi_{s,t}(i) }{\Psi_{s,t}}$$
    where vertices $s,t,i$ are all different from each other

-   $\Psi_{s,t}$ denotes the number of shortest paths (geodesics)
    between vertices $s$ and $t$

-   $\Psi_{s,t}(i)$ denotes the number of shortest paths (geodesics)
    between vertices $s$ and $t$ **that pass through vertex** $i$.

-   The geodesic betweenness $B_n$ of a network is the mean of $B_n(i)$
    over all vertices $i$


In [0]:
betweenness = nx.betweenness_centrality(G)
sorted(betweenness.items(),key=lambda x:-x[1])[:10]

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Mgr_Bienvenu_par_Gustave_Brion.jpg/144px-Mgr_Bienvenu_par_Gustave_Brion.jpg" style="float: right; height: 300px;">




In Betweenness the second most important character is *Myriel*



Is that right?

In [0]:
bokeh_plot(G,betweenness,'Les Miserable - Betweenness')

Betweeness rewards bottlenecks of information as many paths go through them.

Have a look at the plot, can you think why Myriel ends up being the (second) most important?






### Answer



Myriel is the only path between the rest of the network and 7 other characters. To all of the paths between those characters and all other characters go through him. Thus, even through he is not very *central* in the plot, he is very important for information flow in the network. 

It is for this reason that one use of  betweenness is to select the best nodes to remove in a network to split it up :-)

# So who is the most important???? 





<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Gavroche_%28Les_Misérables%29.jpg/162px-Gavroche_%28Les_Misérables%29.jpg" style="float: right; height: 300px;">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Mgr_Bienvenu_par_Gustave_Brion.jpg/144px-Mgr_Bienvenu_par_Gustave_Brion.jpg" style="float: right; height: 300px;">





Well it depends on what is important to your problem. 

Is number of connections important to you? 
- Well then maybe it is **Gavroche** from the Degree Centrality.

Katz Centrality: Are the number of walks of different lengths between nodes important in your network?
- Maybe it is again **Gavroche**

Is the flow of random walkers on your network important to you?
- Maybe it is **Myriel** from the pagerank calculation

Are shortest paths important to you in your network? 
 - Well then maybe it is **Myriel** from the Betweenness calculation.

etc


**Important Questions to consider:**
- What does an edge mean in your network?
- What does a path mean in your network?


**If we/you have some time**

If we/you have some extra time here why not try out another centrality measure.

I would advise trying out closeness centrality (nx.closeness_centrality), but other choices are also good :-)


# Mesoscale Structure - Community Detection/Core-periphery

## Community Detection

Community detection is the process of finding sets of nodes in a network that are densely internally. Algorithms for this process generally find the boundaries of communities by analysing the density of connections between a group of nodes with respect to the density of connections outside of this group. A pair of nodes is more likely to be connected if they are both members of the same community.


<img src="https://github.com/nestauk/im_tutorials/blob/master/img/community_detection.png?raw=true" alt="communities" width=200>

There are [many different types of community detection](https://github.com/benedekrozemberczki/awesome-community-detection). Here we will use the Louvain Method, as there is an actively maintained, easy to use Python implementation, [`python-louvain`](https://python-louvain.readthedocs.io).

It optimises a quantity called modularity:

$$  \sum_{ij} (A_{ij} - \lambda P_{ij}) \delta(c_i,c_j) $$

$A$ - The adjacency matrix

$P_{ij}$ - The expected connection between $i$ and $j$.

$\lambda$ - Resolution parameter

Can use lots of different forms for $P_{ij}$ but the standard one is the so called configuration model:

$P_{ij} = \frac{k_i k_j}{2m}$

In [0]:
# `python-louvain` imports as `community`
import community

To find which community each research topic is in, we apply `best_partition` to our cooccurrence network. We can vary the resolution to change granular the community detection is. We also pass in the name of the edge weight that we want the method to use when determining where community boundaries are.

 **Import Data**

The data for this project is stored as a csv on Amazon Web Services (AWS) S3, a static cloud file storage service. The `im_tutorials` module contains functions for pulling these datasets directly into a dataframe in our notebooks. (Behind the scences wer are using `smart_open` and `pandas` to pull the data)

In [0]:
import pandas as pd
import networkx as nx
from im_tutorials.data import arxiv

In [0]:
arxivData = arxiv.arxiv_sample_2017()

In [0]:
arxivData.head()

Okay so we will now make a network out of this data. 

The simplest network we can make is a co-authorship network, where nodes are authors and there is an edges between a pair of nodes if they have authored a paper together.

First we need to get a unique identifier for an author. For simplicity we will just use name, fine for this demo **but this can be problematic -  many people have the same name**.  

To do this we will use this function:

In [0]:
def formatName(x):
  if 'forenames' in x:
    return x['forenames'] +' '+x['keyname']
  else:
    return 'Unknown' +' '+x['keyname']

To make the network small enough for this session we will only consider:
- The first 40000 papers
- Only papers that have 3 or fewer authors - to avoid cliques!

We will first count the number of co-publications a pair of authors have and then make the graph:

In [0]:


# Count the 
edgeCounter = Counter()
for authorList in arxivData['authors'][:40000]:
    if len(authorList)>3:
      continue
   # Format the authors names
    authorListFormatted = [formatName(x) for x in authorList]
    # Get all pairs of authors
    for author1,author2 in it.combinations(authorListFormatted,2):
      # Use string ordering to give order
      if author1<author2:
        edgeCounter[(author1,author2)]+=1
      else:
        edgeCounter[(author2,author1)]+=1

Generate the graph from the co-publication counts. (Note there are faster ways to do this if the graph is large - see networkx docs)

In [0]:
Garxiv=nx.Graph()
for item in edgeCounter:
  Garxiv.add_edge(item[0],item[1],weight=edgeCounter[item])

As before lets explore our new graph a little:

In [0]:
print(['Num_Nodes', Garxiv.number_of_nodes()])
print(['Num_Edges', Garxiv.number_of_edges()])
print(['Num Trianges', sum(nx.triangles(Garxiv).values())/3])


Lets now check for the number of components.

We could analyse the whole graph (sometimes this is useful) but for this colab on a small google machine it is probably better to use a smaller graph.

Notice that the number of nodes and the number of edges is about equal.

**This means that there is lots of disconnected components that dont really contribute to the network structure**

Thus as before we will focus on the largest connected component (LCC). 

This is the largest set of vertices for which there is a path between any pair of them. 

First we will get all of the connected components:

In [0]:
components = list(nx.connected_components(Garxiv))

Let us look at the histogram of sizes:

In [0]:
pylab.hist([len(x) for x in components])
pylab.yscale('log')

Let us first get the nodes for the largest component

In [0]:
GarxivLCCNodes = max(components,key=lambda x:len(x))

Then generate the graph

In [0]:
GarxivLCC = Garxiv.subgraph(GarxivLCCNodes).copy()

In [0]:
print(['Num_Nodes', GarxivLCC.number_of_nodes()])
print(['Num_Edges', GarxivLCC.number_of_edges()])
print(['Num Trianges', sum(nx.triangles(GarxivLCC).values())/3])

Much smaller and easier to deal with :-). 

Lets have a look at the graph:


In [0]:
bokeh_plot(GarxivLCC,title='Arxiv Network')

Lets now look at the communities:

In [0]:
partition = community.best_partition(GarxivLCC)

How many did we end up with? (The number of communities in modularity optimisation is up to the algorithm - with some caveats!)

In [0]:
max(partition.values())

We can visualise the communities, to see them we will plot each community in a different colour.

To do this we use factor=True as this is a categoricial variable.

In [0]:
bokeh_plot(GarxivLCC,partition,'Co-Authorship',factor=True)

The network seems to split into several nice groups :-).

Some of this is just because the network is so sparse, but we should check what the groups mean :-)

Let us first extract what each of the authors specialise in using the tags on the papers.

For simplicity, rather than the complete tags we will just use the overall subject areas

If you are feeling particularly keen you could also try using the text in the titles:

In [0]:
# Data structure to hold the number of papers of each type.
author2properties = {x:Counter() for x in GarxivLCC}

def get_SubjectArea(x):
   # We are interested in the subject area 
   # (everything before the fullstop)
   # If you have soe time change this function 
   # and see what difference it makes
   return x.split('.')[0]

# This is not the best way to do this
# but for simplicity it is acceptable on such a small dataset
for idx,row in arxivData.head(40000).iterrows():
    authors = row.authors
    if len(authors)>3:
      continue
    authorsFmt = [formatName(x) for x in authors]
    for x in authorsFmt:
      # Filtering only authors in our network
      if x in GarxivLCC:
        for y in row.category_ids:
            author2properties[x][get_SubjectArea(y)]+=1


Lets now compute the percentage of authors that have written a paper in anyone of our areas.

First we will get the number of nodes in each of the communities:

In [0]:
communitySizes = Counter(partition.values())

Then we will compute the 

In [0]:
community2properties = {x:Counter() for x in range(1+max(partition.values()))}
# For each node
for x in partition:
     # Get their group
    group = partition[x]
    # For each subject area they have written a paper in
    for y in author2properties[x]:
        community2properties[group][y.split('.')[0]]+=1
# Normalise by the size of the communities
for x in community2properties:
     for y in community2properties[x]:
        community2properties[x][y]/=communitySizes[x]

Lets check out a given community:

In [0]:
community2properties[1]

The output of the Louvain algorithm is stochastic, so you might get different results :-)

The first time I ran this I obtained:


```
print(community2properties[1])
Counter({'astro-ph': 0.6, 'hep-ex': 0.2, 'hep-ph': 1.0, 'hep-th': 0.4})
```

A community that is clearly related to astro and high energy physics :-). 

There are different ways to validate communities, we don't have time to go through this here, but would be happy to talk about it after. Some options include:

* Hypergeometric test of the labels (there is an over-representation of a labels in a given group) - watch out for multiple comparisons here!.
* Comparsion to a known division using a partition similarity measure (Adjusted Rand Index, Normalised Mutual Information, etc).

Again not enough time, but we can also use community labels for prediction, as we would expect nodes in the same community to behaviour similarly. 

**If we happen to have more time here**

Can you visualise the commnuity categories?

Or you could checkout the other notebook on dynamic community detection (written by George)


# Conclusions

🎈We made it!

In this tutorial, we have constructed and analysed two networks. 

First we analysed the network from Les Miserable and learned about centrality measures. Crucially that it is important to choose your centrality measure carefully.

Second, we briefly look at community detection in a co-authorship network. We constructed the network by considering a filtered list of papers from the arxiv dataset. We extracted author information from each paper, and then used this to form a weighted network.

Using this network we learned about connected components, and then performed community detection in the resulting network, finding that the clusters appear to relate to subject field.


🗺**Where can we go from here?**

As we discussed in the slides, all of these techniques can be used in any network you like. What would make the most sense in your domain/dataset?

How would it be best to construct a network from that data? For this network which centrality measure makes the most sense? What would the communities look like in this dataset, and would they relate to properties of the network?
